In [1]:
import numpy as np
import nibabel as nib
import os
from sklearn import preprocessing

/anaconda3/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
full_data = []
for init in range(16384):
    full_data.append([])

In [3]:
simTB = '../../simTB/'
for im in os.listdir(simTB):
    if im.endswith('.nii'):
        print(im)
        img = nib.load(simTB+im)
        image = img.get_fdata()
        arr = list(image.reshape(16384,400))
        #print(np.mean(arr[0]))
        arr_norm = preprocessing.scale(arr, axis=1)
        #print(np.mean(arr_norm[0]))
        for voxel in range(len(arr_norm)):
            full_data[voxel].extend(arr_norm[voxel])

1.nii
10.nii
11.nii
12.nii
13.nii
14.nii
15.nii
16.nii
17.nii
18.nii
19.nii
2.nii
20.nii
3.nii
4.nii
5.nii
6.nii
7.nii
8.nii
9.nii


In [110]:
full_data = np.asarray(full_data)

In [111]:
full_data.shape

(16384, 8000)

In [112]:
data_full = np.transpose(full_data)
data_full.shape

(8000, 16384)

In [113]:
#data_norm = preprocessing.normalize(data_full, axis=0)
# minmax = preprocessing.MinMaxScaler()
# data_scaled = minmax.fit_transform(data_full)

In [114]:
np.save('simtb_full_std.npy', data_full)

# Now let's apply a mask to the data before formatting it for our network

In [115]:
masknib = nib.load('simexp_MASK.nii')
mask = masknib.get_fdata()

In [116]:
masked_data = []
for init in range(16384):
    masked_data.append([])

In [117]:
n=0
simTB = './simTB/'
for im in os.listdir(simTB):
    if im.endswith('.nii'):
        n +=1
        img = nib.load(simTB+im)
        image = img.get_fdata()
        image = np.reshape(image, (128,128,400))
        brain = np.zeros((128,128,400))
        for ts in range(len(image[0,0,:])):
            #brain[:,:,ts]=np.matmul(image[:,:,ts],np.transpose(mask))
            brain[:,:,ts]=image[:,:,ts]*mask
        masked = nib.Nifti1Image(brain, img.affine)
        nib.save(masked, './simTB_masked/{}.nii'.format(n))
        arr = list(brain.reshape(16384,400))
        arr_norm = preprocessing.scale(arr, axis=1)
        for voxel in range(len(arr_norm)):
            masked_data[voxel].extend(arr_norm[voxel])

In [118]:
masked_data = np.asarray(masked_data)

In [119]:
masked_data.shape

(16384, 8000)

In [120]:
data_masked = np.transpose(masked_data)
data_masked.shape

(8000, 16384)

In [121]:
#data_norm = preprocessing.normalize(data_masked, axis=0)
# minmaxm = preprocessing.MinMaxScaler()
# datam_scaled = minmaxm.fit_transform(data_masked)

In [122]:
np.save('simtb_masked_std.npy', data_masked)